### (Arquivo2) Validação dos dados:

Incluem: aplicacao e selecao de caracteristicas e classificacao dos dados.

Realização das etapas:

- Criar vetores de caracteristicas: VAR, RMS, ...(tempo) e FMD, FMN, ...(frequencia)
- Aplicar 1 metodo de selecao de caracteristicas (exemplo: Select Kbest)
- Classificacao (SVM)

Uma característica é uma propriedade individual mensurável ou característica de um fenômeno que está sendo observado. Em nosso caso de EOG, uma característica pode ser extraída no domínio do tempo ou no domínio da frequência. As características a seguir foram retiradas do artigo *EMG Feature Extraction for Tolerance of White Gaussian Noise* \[1\].

### Domínio do tempo

1. Willison Amplitude (WAMP)

    > $ \sum_{i=1}^{N-1}f(|x_i - x_{i+1}|) $
    
    > $ f(x) = \begin{cases} 1 & \text{if } x \gt threshold \\ 0 & \text{otherwise} \end{cases} $

2. Variance of EMG (VAR)

    > $ \frac{1}{N-1}\sum_{i=1}^{N}x_i^2 $

3. Root Mean Square (RMS)

    > $ \sqrt{\frac{1}{N}\sum_{i=1}^{N}|x_i|^2} $

4. Waveform Length (WL)
    
    > $ \sum_{i=1}^{N-1}|x_{i+1} - x_i| $

5. Zero Crossing (ZC)

    > $ \sum_{i=1}^{N}sgn(x_i) $
    
    > $ sgn(x) = \begin{cases} 1 & \text{if } x_i * x_{i+1} \leq 0 \\ 0 & \text{otherwise} \end{cases} $

### Domínio da frequência

1. Median Frequency (FMD)

    > $ \frac{1}{2}\sum_{j=1}^{M}PSD_j $

2. Mean Frequency (FMN)

    > $ \sum_{j=1}^{M}f_j PSD_j \Big{/} \sum_{j=1}^{M}PSD_j $
    
    > $ f_j = j * SampleRate \Big{/} 2 * M $

3. Modified Median Frequency (MMDF)

    > $ \frac{1}{2}\sum_{j=1}^{M}A_j $
    
    > $ A_j = Amplitude\ do\ espectro\ j $

4. Modified Frequency Mean (MMNF)

    > $ \sum_{j=1}^{M}f_jAj \Big{/} \sum_{j=1}^{M}Aj $


\[1\] Phinyomark, Angkoon & Limsakul, Chusak & Phukpattaranont, P.. (2008). EMG Feature Extraction for Tolerance of White Gaussian Noise.
[Disponível neste link](https://www.researchgate.net/publication/263765853_EMG_Feature_Extraction_for_Tolerance_of_White_Gaussian_Noise)

**Desafio 1**: Descrever as características de acordo com o artigo citado e outros disponíveis relacionados. O que está querendo "ser visto" em cada característica? Qual é o significado matemático de cada uma delas?

### Aplicando as características

É necessário implementar as características, geralmente em formato de funções ou métodos, para que seja possível aplicar tais funções aos dados de entrada e obter as características resultantes. A seguir temos a implementação das características **VAR & RMS** (domínio do tempo) e **FDM & MMDF** (domínio da frequência).

In [1]:
from math import prod
import numpy as np

# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2


# funções de extração de características

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def log_det(x):
    from math import e
    return e ** (np.sum(np.log10(np.abs(x)), axis=-1) /  np.prod(x.shape))

def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

**Tarefa 2:** Implemente todas as características apresentadas neste tutorial em formato de funções. Sinta-se livre também para buscar e implementar características além das apresentadas, citando as fontes de tais características.

### Vetor de características

Ao final da implementação e seleção das características, deve ser escolhida as características e então teremos um vetor com todas elas implementadas.

O vetor de características estará organizado da seguinte forma (exemplo p/ VAR, RMS, RDM e MMDF):

| ID sample | VAR1 | RMS1 | FMD1 | MMDF1 | VAR2 | RMS2 | FMD2 | MMDF2 | Classe |
|:---------:|:----:|:----:|:----:|:-----:|------|------|------|-------|:------:|
|     1     |  v1  |  v1  |  v1  |   v1  | v1   | v1   | v1   | v1    |    0   |
|     2     |  v2  |  v2  |  v2  |   v2  | v2   | v2   | v2   | v2    |    0   |
|    ...    |  ... |  ... |  ... |  ...  | ...  | ...  | ...  | ...   |   ...  |
|     N     |  vN  |  vN  |  vN  |   vN  | vN   | vN   | vN   | vN    |    7   |

### Implementação do vetor

In [2]:
# Carregando dados do prepare.ipynb
x = np.load("datasets/segmented_data_t.npy")
w = np.load("datasets/segmented_data_f.npy")

print(f"Shape dos Vetores originais: Tempo:{x.shape} e Freq:{w.shape}")

# Aplicando as caracteristicas
data_var = var(x)
data_rms = rms(x)

data_fmd = fmd(x)
data_mmdf = mmdf(x)

data_var.shape, data_rms.shape, data_fmd.shape, data_mmdf.shape


Shape dos Vetores originais: Tempo:(56, 2, 33, 64) e Freq:(56, 2, 33, 33)


((56, 2, 33), (56, 2, 33), (56, 2, 33), (56, 2, 33))

In [3]:
# Uniao do vetor de caracteristicas inicial

features = np.array([data_var, data_rms, data_fmd, data_mmdf]) 

# Organizacao das dimensoes
features = features.transpose(1, 3, 2, 0)
print(features.shape)

# Criar vetor de caracteristicas definitivo
features = features.reshape(features.shape[0] * features.shape[1],
                            features.shape[2] * features.shape[3])
print(features.shape)

(56, 33, 2, 4)
(1848, 8)


**Tarefa 3:** Realizar normalização dos dados utilizando ferramentas já conhecidas

**Tarefa 4:** Realizar seleção de características dos dados utilizando ferramentas já conhecidas

In [4]:
# Criacao do vetor de labels

from sklearn.preprocessing import OneHotEncoder

labels_str = ['dir', 'esq', 'cima', 'baixo', 'cima', 'baixo',
              'baixo', 'esq', 'dir', 'baixo', 'dir', 'dir', 'esq', 'cima',
              'baixo', 'cima', 'esq', 'dir', 'cima', 'esq', 'baixo', 'esq',
              'dir', 'esq', 'cima', 'dir', 'cima', 'baixo']

# Transofrmando para numerico
lab_dict = {'dir': 0, 'esq': 1, 'cima': 2, 'baixo': 3}
labels_num = [lab_dict[item] for item in labels_str]

# Criacao do vetor de labels final
y = labels_num * int(features.shape[0] / len(labels_num))
len(y)

1848

**Tarefa 5:** Realizar classificacao **SVM**